In [1]:
import numpy
import cv2
import deepface

In [ ]:
cap = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 25.0, (640,480))

while( cap.isOpened() ):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.flip(frame,1)
        out.write(frame)
        cv2.imshow('frame' , frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [14]:
# Initialize webcam (0 = default camera)
cam = cv2.VideoCapture(0)

# Capture one frame
ret, frame = cam.read()

if ret:
    cv2.imwrite("captured_image.png", frame)  
else:
    print("Failed to capture image.")

cam.release()

In [ ]:
#removed lines: imshow shows pending window with result, waitKey & destroyWindow functionalities to close that pending window
#    cv2.imshow("Captured", frame)         
#    cv2.waitKey(0)                      
#    cv2.destroyWindow("Captured")       

In [14]:
import random
import module_1
module_1.random_nb_call()

2